# **STRIPE**

Time-series forecasting is a crucial machine learning problem in various fields including the stock market, climate, healthcare, business planning, space science, communication engineering and traffic flow. Time-series forecasting is the systematic analysis of historic (past) signal correlations to predict future outcomes. Time-series forecasting can be grouped roughly into two classifications based on the model outputs: probabilistic time-series forecasting and deterministic time-series forecasting. 

STRIPE, a probabilistic time-series forecasting model that blends the deterministic approach and the probabilistic approach to yield sharp, distributed and diverse forecasts. It is named STRIPE, the acronym for Shape and Time Diversity in Probabilistic Forecasting. STRIPE consists in a structured shape and time diversity mechanism based on DPP (Determinantal Point Processes) that is differentiable. This model also has an iterative sampling mechanism to have control over diversity structure. The predictions of STRIPE fit is close to the ground truth, making the model state-of-the-art in the time-series forecasting domain.

To read about it more, please refer [this](https://analyticsindiamag.com/guide-to-stripe-shape-and-time-diversity-in-probabilistic-forecast/) article.

# **Python Implementation**

STRIPE needs a PyTorch environment. GPU is optional but preferred.

Step-1: Install STRIPE

The following command installs STRIPE from its source.

In [ ]:
!git clone https://github.com/vincent-leguen/STRIPE.git

Step-2: Install dependencies

The following commands install the libraries `tslearn` and `properscoring` that are required for training and evaluation of the STRIPE model.

In [ ]:
!pip install tslearn
!pip install properscoring 

Step-3: Create the environment

Following codes change the directory to STRIPE/ to import the modules and classes. Create a PyTorch environment

In [ ]:
%cd STRIPE/
import numpy as np
import torch
import random
from torch.utils.data import DataLoader
import warnings; warnings.simplefilter('ignore')
from data.synthetic_dataset import create_synthetic_dataset_multimodal, SyntheticDataset
from models.models import cVAE, STRIPE, STRIPE_conditional, TestSampler_Sequential
from trainer.trainer import train_model, train_STRIPE, eval_model

Step-4: Configure the device

The following codes check for GPU’s availability and enable a GPU runtime, if available.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
random.seed(0) 

Step-5: Prepare data and set training parameters

The following codes prepare data, split it into train-evaluation sets and declare necessary training parameters. Parameters can be altered suitably based on the device configuration and memory availability.

In [ ]:
batch_size = 100
N = 100
N_input = 20
N_output = 20  
sigma = 0.01
gamma = 0.01
X_train_input,X_train_target,X_test_input,X_test_target = create_synthetic_dataset_multimodal(N,N_input,N_output,sigma)
dataset_train = SyntheticDataset(X_train_input,X_train_target)
dataset_test  = SyntheticDataset(X_test_input,X_test_target)
trainloader = DataLoader(dataset_train, batch_size=batch_size,shuffle=True, num_workers=0)
testloader  = DataLoader(dataset_test, batch_size=10,shuffle=False, num_workers=0)
input_size = 1
rnn_units = 128
nlayers = 1
bidirectional = False
latent_dim = 16
fc_units = 10 

Step-6: Train a Probabilistic model

Train a probabilistic model composed of a variational auto-encoder with the following codes. Here, a cVAE is employed.

In [ ]:
model_dilate = cVAE(input_size,rnn_units,nlayers,bidirectional,latent_dim,fc_units,N_output,device).to(device)
train_model(model_dilate, trainloader, testloader, loss_type='dilate', nsamples=10, learning_rate=0.001, device=device, epochs=501, gamma=gamma, alpha=0.5, print_every=50, eval_every=100, verbose=1) 

Step-7: Train STRIPE-shape network

Train the STRIPE-shape part from end-to-end with the following codes. 

In [ ]:
nshapes = 10
stripe_shape = STRIPE('shape',nshapes, latent_dim, N_output, rnn_units).to(device)
train_STRIPE(cvae=model_dilate, stripe=stripe_shape, trainloader=trainloader, testloader=testloader, device=device, mode_stripe='shape',
    nsamples=nshapes, quality='', diversity_kernel='dtw',  learning_rate=0.001, epochs=16, print_every=2,eval_every=5, alpha=0.5)   

Step-8: Train STRIPE-time network 

Train the STRIPE-time part from end-to-end with the following codes. 

In [ ]:
ntimes = 10
stripe_time = STRIPE_conditional('time',ntimes, latent_dim, N_output, rnn_units).to(device)
train_STRIPE(cvae=model_dilate,stripe=stripe_time, trainloader=trainloader, testloader=testloader, device=device, mode_stripe='time', nsamples=ntimes, quality='', diversity_kernel='tdi', learning_rate=0.001, epochs=1, print_every=16,eval_every=5, alpha=0.5) 

Step-9: Evaluate the model

Evaluate the complete model on the test data based on DILATE loss with the following codes.

In [ ]:
test_sampler = TestSampler_Sequential(model_dilate, stripe_shape, stripe_time)
_,_ = eval_model(test_sampler, testloader,nsamples=10, device=device, gamma=0.01,mode='test_sampler') 

#**Related Articles:**

> * [STRIPE](https://analyticsindiamag.com/guide-to-stripe-shape-and-time-diversity-in-probabilistic-forecast/)

> * [SelfTime](https://analyticsindiamag.com/guide-to-selftime-self-supervised-time-series-representation-learning-framework-with-python-code/)

> * [Giotta Time](https://analyticsindiamag.com/guide-to-giotto-time-a-time-series-forecasting-python-library/)

> * [Facebook Prophet](https://analyticsindiamag.com/comprehensive-guide-to-facebooks-prophet-with-python-code/)

> * [LSTM RNN for Stock Price Prediction](https://analyticsindiamag.com/hands-on-guide-to-lstm-recurrent-neural-network-for-stock-market-prediction/)